For splitting the training1600000 data into shuffled train and test sets
so that fine tuning can be performed easily. Data is stored into json and csv. However only the data from json will be used as the csv encoding edits the text within the tweets.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def strip_all_entities(text):
    entity_prefixes = ['@','#']
    words = []
    for word in str(text).split():
        word = str(word).strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    text = ' '.join(words)
    text = text.replace(' ,', '')
    text = text.replace(' , ', ' ')
    return text

def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    text = text.replace(',#', ' #')
    return text.lower()

In [3]:
from sklearn.model_selection import train_test_split
import re
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

In [0]:
training_path = "drive/My Drive/training1600000.csv"

# Load data to DF
df = pd.read_csv(training_path, header=None, engine='python')
df = df.drop(columns=[1,2,3,4]) # drop these 1: id, 2: datetime, 3: no_query, 4: user
df[6] = df[5].apply(strip_links)
df[7] = df[6].apply(strip_all_entities)
df = df.drop(columns=[5,6])
df.loc[df[0] == 4, 0] = 1

In [0]:
# As we have 1,600,000 tweets. Finetuning is done incrementally, hence I set 
# the number of testing data to 20,000.
df_train = df[0:790000].append(df[800000:1590000], ignore_index=True, sort=True)
df_test = df[790000:800000].append(df[1590000:1600000], ignore_index=True, sort=True)

In [0]:
df_test_shuffled = df_test.sample(frac=1).reset_index(drop=True)
df_train_shuffled = df_train.sample(frac=1).reset_index(drop=True)

In [7]:
df_test_shuffled

,0,7
0,1,eating dinner while harvesting 106 fields of s...
1,0,"i must first do laundry, than study.. i can't ..."
2,1,"funny, but there is a twitter user called mike..."
3,0,i want to see taking back sunday again. two ti...
4,0,ok i tried to sign up like 3 times after putti...
...,...,...
19995,0,ed mcmahon and now farrah fawcett.. sad week
19996,1,work then gym. &gt;17 days til i'm in north ca...
19997,0,just talked to my baby! aww i miss him so much!!!
19998,1,just purchased my first song from itunes. by g...


In [8]:
df_train_shuffled

,0,7
0,1,"give him a chance, it took conan 7 years 2 be ..."
1,0,going to go watch a movie and then go to bed g...
2,1,haaaaaa! i love your ideas
3,1,nyc here i come! just had a cup of coffee with...
4,0,my bape keychain fell off grrrrr
...,...,...
1579995,0,night twitterstars!!! i go to kentucky on wed....
1579996,0,boo i told you it would be too long...we gotta...
1579997,0,omg forreal?! im still in bmore! whens he comi...
1579998,0,deleting deleting deleting - cleared about 2k ...


In [0]:
df_train_shuffled.to_csv('train_shuffled.csv')
df_test_shuffled.to_csv('test_shuffled.csv')
df_train_shuffled.to_json('train_shuffled.json')
df_test_shuffled.to_json('test_shuffled.json')
!cp train_shuffled.csv "drive/My Drive/bert_train_test_csv"
!cp test_shuffled.csv "drive/My Drive/bert_train_test_csv"
!cp train_shuffled.json "drive/My Drive/bert_train_test_json"
!cp test_shuffled.json "drive/My Drive/bert_train_test_json"

In [0]:
train_path = "drive/My Drive/bert_train_test_json/train_shuffled.json"
test_path = "drive/My Drive/bert_train_test_json/test_shuffled.json"

In [0]:
df_train = pd.read_json(train_path, lines=False)
df_test = pd.read_json(test_path, lines=False)

In [0]:
df_test

,0,7
0,0,yes! the clap must be done! i miss the scissor...
1,0,you missed the fireworks over the detroit rive...
2,0,i keep falling asleep.. its too early to liste...
3,0,super rough morning... in so many ways. couldn...
4,1,well hello there yes we have
...,...,...
19995,0,i miss all of my thursday meals kitchen friend...
19996,1,"oh okay, well i did know the first band you me..."
19997,1,says follow me on twitter... and i also follow...
19998,0,i'm afraid patrick swayze is not doing so well...


In [0]:
import os

os.getcwd()
os.listdir()

In [0]:
os.chdir('..')